<h1>Import all libraries and reading explored data into Dataframe</h1>

In [1]:
import re
import io
import time

#General libraries needed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Libraries for data pre-processing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import preprocessing

#Libraries for data pre-processing (Log Loss)
from sklearn.datasets import make_blobs
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss

#For Decision Tree implementation
from scipy.stats import entropy
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

#For KNN implementation
from sklearn.neighbors import KNeighborsClassifier

#For Bagging implementation
from sklearn.ensemble import BaggingClassifier

#For AdaBoost implementation
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

#For Random Forest implementation
from sklearn.ensemble import RandomForestClassifier

#For Baseline implementation
from sklearn.dummy import DummyClassifier

#For Logistic Regression
from sklearn.linear_model import LogisticRegression

#For Ensemble
from sklearn.ensemble import VotingClassifier

#Settings
%matplotlib inline
pd.options.display.float_format = '{:.2f}'.format
np.set_printoptions(threshold=np.nan)
sns.set()

def printModelAccuracy(y_test, y_pred):
    # Find the confusion matrix of the result
    cm = pd.DataFrame(confusion_matrix(y_test, y_pred, labels=[1, 2, 3, 4, 5]), \
        index=['true:1', 'true:2', 'true:3', 'true:4', 'true:5'], 
        columns=['pred:1', 'pred:2', 'pred:3', 'pred:4', 'pred:5'])
    print("Confusion Matrix:")
    print(cm)

    # Find the accuracy and F1 score of the result
    asr = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("Accuracy:", asr)
    print("F1:", f1)
    """
    # Log loss
    score = log_loss(y_test, y_pred)
    print("Log Loss:", score)
    """


<h1>AdaBoost (with Decision Tree)</h1>

Refer to the following links on for detail explanation on the implementation:
- [AdaBoost Classifier SKLearn Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)
- [DataCamp Implementation](https://www.datacamp.com/community/tutorials/adaboost-classifier-python)
- [Setting Learning Rate and N Estimators](https://stats.stackexchange.com/questions/82323/shrinkage-parameter-in-adaboost)
- [Parameter Tuning](https://machinelearningmastery.com/tune-learning-rate-for-gradient-boosting-with-xgboost-in-python/)

*Note that the default AdaBoost implementation in SKLearn is Decision Tree 


### Experiment B
Textual attributes only

In [2]:
# Read from dataframe
X_testB = pd.read_pickle("Experiment B Train Test Split/X_test")
X_trainB = pd.read_pickle("Experiment B Train Test Split/X_train")
y_testB = pd.read_pickle("Experiment B Train Test Split/y_test")
y_trainB = pd.read_pickle("Experiment B Train Test Split/y_train")

print(X_testB.shape)
print(X_trainB.shape)
print(y_testB.shape)
print(y_trainB.shape)

(3912, 7464)
(15648, 7464)
(3912, 1)
(15648, 1)


In [8]:
start = time.time()

#Create the AdaBoost classifier. Default base classifiers is Decision Tree. 
# - n_estimator is the number of base classifiers (i.e. weak learners)
# - learning_rate controls the weight adjustments of each base classifiers. Default is 1
# - learning_rate controls the weight adjustments of each base classifiers. Default is 1
parametersB = {"base_estimator__max_depth" : [9], #list(range(5, 10))
              "base_estimator__criterion" : ["gini"], #["gini", "entropy"]
              "base_estimator__splitter" :   ["random"], #["best", "random"]
              "n_estimators": [300], #[50, 100, 300]
              "learning_rate": [0.001] #[0.001, 0.25, 0.5]
             }

adaboostTreeB = GridSearchCV(AdaBoostClassifier(DecisionTreeClassifier()), cv=5, param_grid=parametersB)

#Fit the training feature Xs and training label Ys
adaboostTreeB.fit(X_trainB, y_trainB.values.ravel())

#Use the trained model to predict the test data
y_predB = adaboostTreeB.predict(X_testB)

# Find the confusion matrix, the accuracy, and F1 score of the result
printModelAccuracy(y_testB, y_predB)
print("Best Parameters:",adaboostTreeB.best_params_)

# meansB = adaboostTreeB.cv_results_['mean_test_score']
# stdsB = adaboostTreeB.cv_results_['std_test_score']
# paramsB = adaboostTreeB.cv_results_['params']
# for mean, stdev, param in zip(meansB, stdsB, paramsB):
#     print("%f (%f) with: %r" % (mean, stdev, param))

end = time.time()
print(end - start)

Confusion Matrix:
        pred:1  pred:2  pred:3  pred:4  pred:5
true:1     380     146     190      53      20
true:2     320     167     234      52      29
true:3     298      93     265      81      49
true:4     261      82     195     146     115
true:5     210      60     126     139     201
Accuracy: 0.2962678936605317
F1: 0.2962678936605317
Best Parameters: {'base_estimator__criterion': 'gini', 'base_estimator__max_depth': 9, 'base_estimator__splitter': 'random', 'learning_rate': 0.001, 'n_estimators': 300}
16911.255299806595
